In [1]:
!pip install import_ipynb==0.1.3
!pip install chess

You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [2]:
import chess

import import_ipynb
import Game
from MinimaxAlgorithm import make_move_minimax, make_move_minimax_memoization
from RandomAlgorithm import make_random_move
from AlphaBetaAlgorithm import make_move_alphabeta, make_move_alphabeta_iterative_deepening




importing Jupyter notebook from Game.ipynb
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
importing Jupyter notebook from MinimaxAlgorithm.ipynb
importing Jupyter notebook from Util.ipynb
importing Jupyter notebook from Constants.ipynb
importing Jupyter notebook from RandomAlgorithm.ipynb
importing Jupyter notebook from AlphaBetaAlgorithm.ipynb


In [3]:
#game.board.outcome()

In [4]:
# Random
# Minimax
    # MinimaxMemo
# AlphaBeta
    # AlphaBeta_IterativeDeepening

random_minimax = Game.Game(make_random_move, make_move_minimax)
random_alpha = Game.Game(make_random_move, make_move_alphabeta)
minimax_alpha = Game.Game(make_move_minimax, make_move_alphabeta)

random_minimemo = Game.Game(make_random_move, make_move_minimax_memoization)
random_alphadeep = Game.Game(make_random_move, make_move_alphabeta_iterative_deepening)
minimemo_alpha = Game.Game(make_move_minimax_memoization, make_move_alphabeta)
minimemo_alphadeep = Game.Game(make_move_minimax_memoization, make_move_alphabeta_iterative_deepening)
alphadeep_mini = Game.Game(make_move_alphabeta_iterative_deepening, make_move_minimax)


mini_minimemo = Game.Game(make_move_minimax, make_move_minimax_memoization)
alpha_alphadeep = Game.Game(make_move_alphabeta, make_move_alphabeta_iterative_deepening)

In [ ]:
random_minimax.play()

In [ ]:
random_alpha.play()

In [ ]:
minimax_alpha.play()

In [ ]:
random_minimemo.play()

In [ ]:
random_alphadeep.play()

In [5]:
minimemo_alpha.play()

In [ ]:
minimemo_alphadeep.play()

In [ ]:
alphadeep_mini.play()

In [ ]:
mini_minimemo.play()

In [ ]:
alpha_alphadeep.play()

In [13]:
#print(game.move_times)
#white_d = sum(game.move_times[chess.WHITE]) / len(game.move_times[chess.WHITE])
#black_d = sum(game.move_times[chess.BLACK]) / len(game.move_times[chess.BLACK])

#print(white_d)
#print(black_d)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=d6ce9acd-52c5-4422-904d-8424da19408b' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>